# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [25]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs
from shapely.ops import nearest_points
from turfpy.misc import nearest_point_on_line

In [2]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

In [3]:
#encontra a latitude e longitude e retorna 

def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None
    
    
    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road
    
    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]
        
    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)

    # Apply km inversion correction
    if invert_km:
        
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
        
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)
    
    return Point(point.x, point.y), road

In [4]:
#verifica se o km está dentro da rodovia

def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [5]:
def get_connected_reference(company, resource, key, value):
    if value is not None:
        value = value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [11]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [33]:
print(road)

2059: ERS-122


 Verificando se está no ambiente de produção

In [13]:
!cat .env

cat: .env: No such file or directory


Escolhendo nome do arquivo Excel para carregar o Inventário

In [14]:
filename='kartado_Bueiro_Pt1'

Escolhendo a aba da planilha do Excel

In [15]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [16]:
print(ws['A1'].value)

Código do Inventário para vinculo com apontamento


Carregando os valores no script

In [27]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

Caminho pasta Aws Images

In [28]:
ficha_path = r'csg_drenagem_2024/'

In [32]:
for a in values:
    a['img'] = []
#         print(a)
    for i in range(1,11):
        # Imagem Geral
        if a.get('Foto_'+str(i)) is not None and a.get('Foto_'+str(i)) is not '':

            a['img'].append({
                'foto': ficha_path+a.get('Foto_'+str(i)),
                'data_foto': a.get('Data Foto_'+str(i)),
                'tipo_foto': a.get('Tipo Foto_'+str(i)),
                'des_foto': a.get('Descrição Foto_'+str(i))
            })

<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_7971/1114558654.py:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if a.get('Foto_'+str(i)) is not None and a.get('Foto_'+str(i)) is not '':


Definindo valores da importação

In [21]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Bueiros
occurrence_type=OccurrenceType.objects.get(uuid='083d75c7-0bf7-4a0b-bdb8-4f43f10f9266')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='7e50d5c5-4019-4165-ab39-39782fec0118')
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Bueiro - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 7e50d5c5-4019-4165-ab39-39782fec0118: Egati>)

In [ ]:
# road=Road.objects.get(company=company,name__icontains="470")
# items=list(road.marks.items())
# first_km=items[0][1]['km']
# last_km=items[-1][1]['km']
# print(first_km,last_km)

In [22]:
for a in values:
    a['Latitude'] = '-'+str(a['Latitude']) if '-' not in str(a['Latitude']) else str(a['Latitude'])
    a['Longitude'] = '-'+str(a['Longitude']) if '-' not in str(a['Longitude']) else str(a['Longitude'])
    if a['Lado montante'] == 'Central':
        a['Lado montante'] = "Centro"
    if a['Lado montante'] == 'Esquedo':
        a['Lado montante'] = "Esquerdo"

In [37]:
for a in values:
    if a['Faixa'] == None:
        a['Faixa']= "Não se aplica"

In [33]:
values[0]

{'Código do Inventário para vinculo com apontamento': 'BU 240 RS 000+001 D 1',
 'Latitude': -29.73054845,
 'Longitude': -51.14926918,
 'km': '000+001',
 'km final': None,
 'km de Projeto': '000+001',
 'km final de Projeto': None,
 'Status': 'Executado',
 'Equipe': 'Egati',
 'Encontrado em': None,
 'Executado em': None,
 'Prazo': None,
 'Sentido': 'Decrescente',
 'Classe': 'Bueiro',
 'Faixa': None,
 'Rodovia': 'ERS-240',
 'Tipo': 'SIMPLES',
 'Lado montante': None,
 'Estado de conservação': 'Bom',
 'Dispositivo de entrada': None,
 'Dispositivo de saída': None,
 'Dimensão': '1,2x1,2',
 'Extensão': None,
 'Observações': None,
 'Monitoração': None,
 'Recurso_1': None,
 'Quantidade_1': None,
 'Recurso_2': None,
 'Quantidade_2': None,
 'Recurso_3': None,
 'Quantidade_3': None,
 'Recurso_4': None,
 'Quantidade_4': None,
 'Recurso_5': None,
 'Quantidade_5': None,
 'Recurso_6': None,
 'Quantidade_6': None,
 'Recurso_7': None,
 'Quantidade_7': None,
 'Recurso_8': None,
 'Quantidade_8': None,
 'Re

In [34]:
for i, a in enumerate(values):
    road_name= a['Rodovia']
    point, road=get_road_coordinates(road_name, float(a['km'].replace('+', '.')), str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())), company)
#     if road == None and road_name != road.name:
    print(road_name, road)

ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 20

ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
ERS-240 2060: ERS-240
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 2044: RSC-287
RSC-287 20

ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122
ERS-122 2059: ERS-122


KeyboardInterrupt: 

Criando itens de Inventário

In [36]:
values[0]

{'Código do Inventário para vinculo com apontamento': 'BU 240 RS 000+001 D 1',
 'Latitude': -29.73054845,
 'Longitude': -51.14926918,
 'km': '000+001',
 'km final': None,
 'km de Projeto': '000+001',
 'km final de Projeto': None,
 'Status': 'Executado',
 'Equipe': 'Egati',
 'Encontrado em': None,
 'Executado em': None,
 'Prazo': None,
 'Sentido': 'Decrescente',
 'Classe': 'Bueiro',
 'Faixa': None,
 'Rodovia': 'ERS-240',
 'Tipo': 'SIMPLES',
 'Lado montante': None,
 'Estado de conservação': 'Bom',
 'Dispositivo de entrada': None,
 'Dispositivo de saída': None,
 'Dimensão': '1,2x1,2',
 'Extensão': None,
 'Observações': None,
 'Monitoração': None,
 'Recurso_1': None,
 'Quantidade_1': None,
 'Recurso_2': None,
 'Quantidade_2': None,
 'Recurso_3': None,
 'Quantidade_3': None,
 'Recurso_4': None,
 'Quantidade_4': None,
 'Recurso_5': None,
 'Quantidade_5': None,
 'Recurso_6': None,
 'Quantidade_6': None,
 'Recurso_7': None,
 'Quantidade_7': None,
 'Recurso_8': None,
 'Quantidade_8': None,
 'Re

In [41]:
road_name

'ERS-122'

In [45]:
objects=[]


for index, a in enumerate(tqdm(values)):
#     print(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']))
    km = float(a['km'].replace('+', '.'))
    end_km = float(a['km final'].replace('+', '.')) if a.get('km final') is not None else km
    road_name=a['Rodovia']
    point, road=get_road_coordinates(road_name, float(a['km'].replace('+', '.')), str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())), company)
    if a['Rodovia'] != road.name:
        print(a['Rodovia'],road.name)
    try:

        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
                    end_km_manually_specified=True if a['km final'] is not None else False,
                    end_km=float(end_km) if a['km final'] is not None else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'type': a['Tipo'],
                        'side': get_value('Lado montante', occurrence_type, a['Lado montante']) if a['Lado montante'] is not None else None,
#                         'side': a['Lado montante'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data de fabricação'] and type(a['Data de fabricação']) is datetime.datetime else None,
                        'state': get_value('Estado de conservação', occurrence_type, a['Estado de conservação'].capitalize() if a.get('Estado de conservação') else a['Estado de Conservação'].capitalize()) if a.get('Estado de conservação') is not None or a.get('Estado de Conservação') is not None else None,
                        'dimensao': str(a['Dimensão']),
                        'extension': a['Extensão'],
                        'dispentrada': a['Dispositivo de entrada'],
                        'dispsaida': a['Dispositivo de saída'],                                                     
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(obj.tese)  # Acessando o atributo form_data de cada objeto na lista
        

  0%|          | 0/999 [00:00<?, ?it/s]

Salvando itens de Inventário no sistema e imprimindo os seriais

In [46]:
objects[0][0].__dict__,objects[0][0].__dict__

({'_state': <django.db.models.base.ModelState at 0x792dc5897940>,
  'uuid': UUID('beb424a0-fb9a-4b76-b9d0-42ac01a6177a'),
  'number': '',
  'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
  'road_name': 'ERS-240',
  'road_id': 2060,
  'km': 0.001,
  'end_km': None,
  'km_reference': None,
  'project_km': 0,
  'project_end_km': None,
  'point': <Point object at 0x792dc5bf1990>,
  'direction': '2',
  'lane': '99',
  'track': None,
  'branch': None,
  'address': {},
  'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
  'firm_id': UUID('7e50d5c5-4019-4165-ab39-39782fec0118'),
  'occurrence_type_id': UUID('083d75c7-0bf7-4a0b-bdb8-4f43f10f9266'),
  'form_data': {'type': 'SIMPLES',
   'side': None,
   'state': '1',
   'dimensao': '1,2x1,2',
   'extension': None,
   'dispentrada': None,
   'dispsaida': None},
  'form_metadata': {},
  'executed_at': None,
  'created_at': None,
  'updated_at': None,
  'found_at': datetime.datetime(2024, 4, 18, 17, 7, 14, 898567, tzinfo=<

In [ ]:
# for reporting, images in tqdm(objects):
#     reporting.save()
#     print(reporting.number)

Anexando as imagens

In [26]:
fotos_apt=[]
apt=[]

for reporting, images in tqdm(objects):
#    reporting.save()
    apt.append(reporting)
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.get('des_foto') if image.get('des_foto') else 'CSG',
            datetime=image.get('data_foto', datetime.datetime(2000, 1, 1, 0, 0)) if isinstance(image.get('data_foto'), datetime.datetime) else datetime.datetime(2000, 1, 1, 0, 0),
            kind=image.get('tipo_foto', 'outro').lower() if image.get('tipo_foto') is not None else 'outro',
            km=reporting.km,
            is_shared=False,
            point=reporting.point,
            upload=image.get('foto')
        )
        try:
            reporting_file.upload
            fotos_apt.append(reporting_file)
#            reporting_file.save()
        except Exception as e:
            print(reporting, reporting_file, {objects.eu(e)})

  0%|          | 0/999 [00:00<?, ?it/s]

In [ ]:
# for reporting, images in tqdm(objects):
#     for image in images:
#         if hasattr(image, 'name'):
# #             print('O objeto tem o atributo nome!')
#             pass
#         else:
# #             print('O objeto não tem o atributo nome.')
#             image.name="CSG"
#             print(image.name)

In [ ]:
len(apt),len(fotos_apt),fotos_apt[4].__dict__

In [25]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/999 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10101
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10102
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10103
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10104
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10105
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10106
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10107
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10108
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10174
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10175
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10176
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10177
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10178
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10179
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10180
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10181
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Found KM on Road 2059: ERS-122
CSG-Inv-2024.10246
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10247
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10248
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10249
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10250
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10251
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10252
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10253
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10254
Searching KM on direction...
Searching KM with

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10320
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10321
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10322
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10323
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10324
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10325
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10326
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10327
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

...

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10758
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10759
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10760
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10761
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10762
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10763
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10764
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10765
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10831
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10832
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10833
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10834
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10835
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10836
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10837
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10838
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

Found KM on Road 2059: ERS-122
CSG-Inv-2024.10904
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10905
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10906
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10907
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10908
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10909
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10910
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10911
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10912
Searching KM on direction...
Searching KM with

CSG-Inv-2024.10978
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10979
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10980
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10981
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10982
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10983
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10984
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10985
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.10986
Searching KM on direction...
Searching KM without direction...
Found KM on Ro

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11052
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11053
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11054
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11055
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11056
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11057
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11058
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2024.11059
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CS

In [ ]:
apontamentos = bulk_update_with_history(apt, Reporting, use_django_bulk=True, batch_size=100, user=user)

In [27]:
fot = bulk_create_with_history(fotos_apt, ReportingFile, default_user=user)

/home/joaonitzke/.cache/pypoetry/virtualenvs/apps-t2o7wolH-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/joaonitzke/.cache/pypoetry/virtualenvs/apps-t2o7wolH-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalReportingFile.datetime received a naive datetime (2000-01-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
